<a href="https://colab.research.google.com/github/humanet1603/krx_using_api/blob/main/VAR_MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### FRED API를 활용하여 미국경제 지표 가져오기

In [ ]:
import requests
import pandas as pd
import json
import time

class FredAPI:
    def __init__(self, api_key):
        self.api_key = api_key
        self.base_url = 'https://api.stlouisfed.org/fred/'

    def get_series(self, series_id, start_date=None, end_date=None):
        url = f"{self.base_url}series/observations"
        params = {
            'series_id': series_id,
            'api_key': self.api_key,
            'file_type': 'json'
        }
        if start_date:
            params['observation_start'] = start_date
        if end_date:
            params['observation_end'] = end_date

        response = requests.get(url, params=params)
        data = response.json()

        if 'observations' in data:
            observations = data['observations']
            df = pd.DataFrame(observations)
            df['date'] = pd.to_datetime(df['date'])
            df.set_index('date', inplace=True)
            df['value'] = df['value'].astype(float)
            return df[['value']]
        else:
            raise Exception("Error fetching data from FRED API: ", data.get('error_message', 'Unknown error'))

    def get_series_info(self, series_id):
        url = f"{self.base_url}series"
        params = {
            'series_id': series_id,
            'api_key': self.api_key,
            'file_type': 'json'
        }

        response = requests.get(url, params=params)
        data = response.json()

        if 'seriess' in data:
            return data['seriess'][0]
        else:
            raise Exception("Error fetching series info from FRED API: ", data.get('error_message', 'Unknown error'))

# 사용 예제
if __name__ == "__main__":
    # FRED API 키 설정
    api_key = 'YOUR_API_KEY'
    fred_api = FredAPI(api_key)

    # 특정 경제 지표 데이터 불러오기 (예: GDP)
    series_id = 'GDP'
    start_date = '2000-01-01'
    end_date = '2024-01-01'
    gdp_data = fred_api.get_series(series_id, start_date, end_date)
    print(gdp_data)

    # 경제 지표 정보 불러오기
    series_info = fred_api.get_series_info(series_id)
    print(json.dumps(series_info, indent=4))


In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.api import VAR
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller

# 데이터 불러오기 (예시: FRED에서 미국 경제 지표 데이터를 불러옴)
data = pd.read_csv('US_economic_data.csv', index_col=0, parse_dates=True)

# 데이터 전처리
data = data.dropna()

# 정상성 확인 및 차분 적용
for column in data.columns:
    result = adfuller(data[column])
    if result[1] > 0.05:
        data[column] = data[column].diff().dropna()

# VAR 모델 적합
model = VAR(data)
results = model.fit(maxlags=15, ic='aic')

# 예측
lag_order = results.k_ar
forecast = results.forecast(data.values[-lag_order:], steps=10)
forecast_df = pd.DataFrame(forecast, index=pd.date_range(start=data.index[-1], periods=10, freq='M'), columns=data.columns)

# 예측 결과 시각화
forecast_df.plot()
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'US_economic_data.csv'